In [1]:
import os
import pandas as pd

In [2]:
os.chdir("/fs/scratch/PAS0439/Ming/virome_ecology_core_prkaryotes/results/08_inStrain_rug4941/")

In [3]:
df = pd.read_csv("repre_genomes_instrain.genes.faa_vs_dbCAN_v11.csv")

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
cazy = df[["column0", "column2", "column3", "column5", "column6", "column15","column16","column17","column18"]]
cazy.columns = ["CAZyme", "qlen", "gene", "slen", "E_value", "qstart", "qend", "sstart", "send"]

In [6]:
for index, row in cazy.iterrows():
    cazy.loc[index, "hmm_coverage"] = (cazy.loc[index, "qend"] - cazy.loc[index, "qstart"]) / cazy.loc[index, "qlen"]

/tmp/slurmtmp.16452030/ipykernel_189149/1822478317.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cazy.loc[index, "hmm_coverage"] = (cazy.loc[index, "qend"] - cazy.loc[index, "qstart"]) / cazy.loc[index, "qlen"]


In [7]:
cazy_ready = cazy[["CAZyme", "gene", "E_value", "hmm_coverage"]]

In [8]:
cazy_ready.loc[:,"CAZyme"] = list(cazy_ready.CAZyme.apply(lambda x: x.split(".hmm")[0]))

/tmp/slurmtmp.16452030/ipykernel_189149/234930382.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cazy_ready.loc[:,"CAZyme"] = list(cazy_ready.CAZyme.apply(lambda x: x.split(".hmm")[0]))


In [9]:
cazy_ready_filtered = cazy_ready.query('E_value == E_value')

In [10]:
cazy_hits = cazy_ready_filtered.loc[cazy_ready_filtered.groupby("gene")["E_value"].idxmin()]

In [11]:
gh = cazy_hits[cazy_ready_filtered.CAZyme.str.contains("GH")]

/tmp/slurmtmp.16452030/ipykernel_189149/2535724748.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  gh = cazy_hits[cazy_ready_filtered.CAZyme.str.contains("GH")]


In [12]:
gh_ready = gh.query('E_value < 1e-15').query('hmm_coverage > 0.35').reset_index(drop = True)

In [13]:
gh_ready

,CAZyme,gene,E_value,hmm_coverage
0,GH109,Archaea_Methanomicrobium_mobile_DSM_1539_scaff...,9.600000e-23,0.385965
1,GH3,Bacteria_Bifidobacterium_bifidum_Calf96_scaffo...,4.400000e-39,0.962963
2,GH109,Bacteria_Bifidobacterium_bifidum_Calf96_scaffo...,3.900000e-16,0.395990
3,GH13_30,Bacteria_Bifidobacterium_bifidum_Calf96_scaffo...,9.600000e-163,0.997260
4,GH77,Bacteria_Bifidobacterium_bifidum_Calf96_scaffo...,1.200000e-136,0.975709
...,...,...,...,...
32516,GH36,T546DRAFT_scaffold00022.22_18,4.200000e-81,0.819767
32517,GH18,T546DRAFT_scaffold00027.27_6,5.700000e-25,0.672297
32518,GH11,T546DRAFT_scaffold00028.28_2,1.100000e-69,0.983051
32519,GH13_31,T546DRAFT_scaffold00028.28_6,9.800000e-133,0.997135


In [14]:
gh_ready.to_csv("dbcan_out_strigent_RUG4941.csv", index = None)

In [15]:
gh_ready[["gene", "CAZyme"]].rename(columns = {"CAZyme":"anno"}).to_csv("geneAnnotations_CAZyme.csv", index = None)